In [54]:
from dotenv import load_dotenv
import os
import requests
load_dotenv()
def get_aqi_str(aqi_key:str)->str:
    url = f'https://data.moenv.gov.tw/api/v2/aqx_p_432?api_key={aqi_key}&limit=1000&sort=ImportDate%20desc&format=JSON'
    try:
        r = requests.get(url)
        r.raise_for_status()
    except Exception as e:
        print(f"發生錯誤:{e}")
    return r.text
aqi_data=get_aqi_str(aqi_key=os.environ["AQI_KEY"])

In [58]:
aqi_data

'{\n    "fields": [\n        {\n            "id": "sitename",\n            "type": "text",\n            "info": {\n                "label": "測站名稱"\n            }\n        },\n        {\n            "id": "county",\n            "type": "text",\n            "info": {\n                "label": "縣市"\n            }\n        },\n        {\n            "id": "aqi",\n            "type": "text",\n            "info": {\n                "label": "空氣品質指標"\n            }\n        },\n        {\n            "id": "pollutant",\n            "type": "text",\n            "info": {\n                "label": "空氣污染指標物"\n            }\n        },\n        {\n            "id": "status",\n            "type": "text",\n            "info": {\n                "label": "狀態"\n            }\n        },\n        {\n            "id": "so2",\n            "type": "text",\n            "info": {\n                "label": "二氧化硫（ppb）"\n            }\n        },\n        {\n            "id": "co",\n            "type": "text"

In [5]:
r.text


'{\n    "fields": [\n        {\n            "id": "sitename",\n            "type": "text",\n            "info": {\n                "label": "測站名稱"\n            }\n        },\n        {\n            "id": "county",\n            "type": "text",\n            "info": {\n                "label": "縣市"\n            }\n        },\n        {\n            "id": "aqi",\n            "type": "text",\n            "info": {\n                "label": "空氣品質指標"\n            }\n        },\n        {\n            "id": "pollutant",\n            "type": "text",\n            "info": {\n                "label": "空氣污染指標物"\n            }\n        },\n        {\n            "id": "status",\n            "type": "text",\n            "info": {\n                "label": "狀態"\n            }\n        },\n        {\n            "id": "so2",\n            "type": "text",\n            "info": {\n                "label": "二氧化硫（ppb）"\n            }\n        },\n        {\n            "id": "co",\n            "type": "text"

In [7]:
Json_data

{'fields': [{'id': 'sitename', 'type': 'text', 'info': {'label': '測站名稱'}},
  {'id': 'county', 'type': 'text', 'info': {'label': '縣市'}},
  {'id': 'aqi', 'type': 'text', 'info': {'label': '空氣品質指標'}},
  {'id': 'pollutant', 'type': 'text', 'info': {'label': '空氣污染指標物'}},
  {'id': 'status', 'type': 'text', 'info': {'label': '狀態'}},
  {'id': 'so2', 'type': 'text', 'info': {'label': '二氧化硫（ppb）'}},
  {'id': 'co', 'type': 'text', 'info': {'label': '一氧化碳（ppm）'}},
  {'id': 'o3', 'type': 'text', 'info': {'label': '臭氧（ppb）'}},
  {'id': 'o3_8hr', 'type': 'text', 'info': {'label': '臭氧8小時移動平均（ppb）'}},
  {'id': 'pm10', 'type': 'text', 'info': {'label': '懸浮微粒（μg/m3）'}},
  {'id': 'pm2.5', 'type': 'text', 'info': {'label': '細懸浮微粒（μg/m3）'}},
  {'id': 'no2', 'type': 'text', 'info': {'label': '二氧化氮（ppb）'}},
  {'id': 'nox', 'type': 'text', 'info': {'label': '氮氧化物（ppb）'}},
  {'id': 'no', 'type': 'text', 'info': {'label': '一氧化氮（ppb）'}},
  {'id': 'wind_speed', 'type': 'text', 'info': {'label': '風速（m/sec）'}},
  {'

In [59]:
from pydantic import BaseModel
class AQI(BaseModel):
    resource_id:str
    include_total:bool
    total:int
    resource_format:str
    limit:int
    offset:int

aqi:AQI = AQI.model_validate_json(aqi_data)
aqi

AQI(resource_id='8d2f907f-bbb4-4fdf-8f08-8eabae15da45', include_total=True, total=85, resource_format='object', limit=1000, offset=0)

In [24]:
#Serializing
#把pydantic的實體轉換為pyhon的資料結構
aqi.model_dump()

{'resource_id': '8d2f907f-bbb4-4fdf-8f08-8eabae15da45',
 'include_total': True,
 'total': 84,
 'resource_format': 'object',
 'limit': 1000,
 'offset': 0}

In [60]:
#Serializing
#把pydantic的實體轉換為json格式的字串

aqi.model_dump_json()

'{"resource_id":"8d2f907f-bbb4-4fdf-8f08-8eabae15da45","include_total":true,"total":85,"resource_format":"object","limit":1000,"offset":0}'

In [61]:
from pydantic import BaseModel,Field,field_validator,field_serializer
from datetime import datetime

class Record(BaseModel):
    站點: str = Field(alias='sitename')
    縣市: str = Field(alias='county')
    日期: datetime = Field(alias='datacreationdate')
    aqi: float | None 
    狀態: str = Field(alias='status')
    pm25: float | None = Field(alias='pm2.5')
    緯度: float = Field(alias='latitude')
    經度: float = Field(alias='longitude')

    @field_validator("pm25","aqi",mode='before')
    @classmethod
    def pm25_for_empty_string(cls,v:str)->float:
        if v == '':
            return None
        else:
            return float(v)
    
    @field_serializer('緯度','經度')
    def four_digits(self,value:float)->float:
        return round(value,ndigits=4)
    
    @field_serializer('日期')
    def datetime_to_string(self,date:datetime) -> str:
        return date.strftime('%Y:%m:%d %H:%M:00')


class AQI(BaseModel):
    records: list[Record]

aqi = AQI.model_validate_json(aqi_data)

ValidationError: 85 validation errors for AQI
records.0.datacreationdate
  Field required [type=missing, input_value={'sitename': '基隆', 'c....129167', 'siteid': '1'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
records.1.datacreationdate
  Field required [type=missing, input_value={'sitename': '汐止', 'c...5.06624', 'siteid': '2'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
records.2.datacreationdate
  Field required [type=missing, input_value={'sitename': '萬里', 'c....179667', 'siteid': '3'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
records.3.datacreationdate
  Field required [type=missing, input_value={'sitename': '新店', 'c....977222', 'siteid': '4'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
records.4.datacreationdate
  Field required [type=missing, input_value={'sitename': '土城', 'c..

In [ ]:
aqi_data:list[dict]=aqi.model_dump()['records']

[{'站點': '基隆',
  '縣市': '基隆市',
  'aqi': 28.0,
  '狀態': '良好',
  'pm25': 6.0,
  '緯度': 25.129167,
  '經度': 121.760056},
 {'站點': '汐止',
  '縣市': '新北市',
  'aqi': 28.0,
  '狀態': '良好',
  'pm25': 6.0,
  '緯度': 25.06624,
  '經度': 121.64081},
 {'站點': '萬里',
  '縣市': '新北市',
  'aqi': 31.0,
  '狀態': '良好',
  'pm25': 10.0,
  '緯度': 25.179667,
  '經度': 121.689881},
 {'站點': '新店',
  '縣市': '新北市',
  'aqi': 33.0,
  '狀態': '良好',
  'pm25': 8.0,
  '緯度': 24.977222,
  '經度': 121.537778},
 {'站點': '土城',
  '縣市': '新北市',
  'aqi': 30.0,
  '狀態': '良好',
  'pm25': 9.0,
  '緯度': 24.982528,
  '經度': 121.451861},
 {'站點': '板橋',
  '縣市': '新北市',
  'aqi': 34.0,
  '狀態': '良好',
  'pm25': 7.0,
  '緯度': 25.012972,
  '經度': 121.458667},
 {'站點': '新莊',
  '縣市': '新北市',
  'aqi': 32.0,
  '狀態': '良好',
  'pm25': 10.0,
  '緯度': 25.037972,
  '經度': 121.4325},
 {'站點': '菜寮',
  '縣市': '新北市',
  'aqi': 33.0,
  '狀態': '良好',
  'pm25': 8.0,
  '緯度': 25.06895,
  '經度': 121.481028},
 {'站點': '林口',
  '縣市': '新北市',
  'aqi': 31.0,
  '狀態': '良好',
  'pm25': 3.0,
  '緯度': 25.07798949,
  '經度